In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
# from matplotlib import pyplot as plt
from PIL import Image
import os
import shutil
import copy
import re
import time

In [20]:
""" 我应当做的是分别加载遮挡&光照不同情况的文件夹，手动删掉空文件夹，将剩余文件夹的文件夹名、图片数量，识别率保存到List，在保存到DF，CSV """
path=r"./images/遮挡&光照/后腿遮挡/"
temp=os.listdir(path)
dirPath=[]
for i in range(len(temp)):
    dirPath.append(path+temp[i]+"/")
""" 保存好对应不同品种文件夹的路径后，创建DF """
dfResult=pd.DataFrame(columns=["breedName","path", "imagesNumber","predsRate"])
dfResult["breedName"]=temp
dfResult["path"]=dirPath
""" 统计不同文件夹的文件数量 """
seek=0
for ele in dirPath:
    count=0
    for root,dirs,files in os.walk(ele):
        for file in files:
            count+=1
    dfResult.at[seek,"imagesNumber"]=count
    seek+=1
""" 接下来是计算不同种类的识别率 """
nowClassToIndex = {
    x: i for i, x in enumerate(temp)
}
"""
IndexToClass = {
    i: x for i, x in enumerate(temp)
}
 """
dfAllFilePath = pd.read_csv("./all_image_path.csv")
classToIndex = {
    x: i for i, x in enumerate(dfAllFilePath["breed"].unique())
}
indexToClass = {
    i: x for i, x in enumerate(dfAllFilePath["breed"].unique())
}
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)
imageTransform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])
model=torch.load("./test4_resnet_epochnumber_30.pkl")
model.cuda()
print()

In [21]:
""" 遍历每个文件夹，构建每个文件的路径，按照路径加载图像，传入网络，如果索引对应的名称等于文件夹名称，预测正确数+1，然后预测正确数除以文件夹图片总数，得出识别率，识别率填入DF """
for ele in dirPath:
    breedName=ele.split("/")[-2]
    label=classToIndex.get(breedName)
    corrects=0
    for root,dirs,files in os.walk(ele):
        for file in files:
            imagePath=ele+file
            # print(imagePath)
            image=Image.open(imagePath)
            if image.mode !="RGB":
                image = image.convert("RGB")
            image=imageTransform(image)
            image=torch.unsqueeze(image,0)
            image=image.float().cuda()
            output=model(image)
            _, preds = torch.max(output.data, dim=1)
            preds=preds.item()
            """ if breedName=="Dilute Calico":
                predsBreed=indexToClass.get(preds)
                print(predsBreed) """
            
            if label==preds:
                corrects+=1
    index=nowClassToIndex.get(breedName)
    # print(index)
    imagesNumber=dfResult.at[index,"imagesNumber"]
    # print(imagesNumber)
    predsRate=corrects/imagesNumber
    predsRate=round(predsRate,2)
    dfResult.at[index,"predsRate"]=predsRate

In [22]:
dfResult.to_csv(path+"结果.csv")

In [2]:
""" 遮挡文件夹已经划分好，构建每个图片的路径、品种 """
#统计不同文件夹下文件的个数
path=r"./images/test_dateset/"
temp=os.listdir(path)
dfBase=pd.DataFrame(temp)
dfBase=dfBase.rename(columns={0:"breed"})
dfBase["dir"]=""
dfBase["count"]=0
#填入不同文件夹的路径
seek=0
for root,dirs,files in os.walk(path):
    for ele in dirs:
        dfBase.loc[seek,"dir"]=os.path.join(path,ele)
        seek+=1
#遍历不同文件夹，计算不同文件夹下的图片文件数量
seek=0
for ele in dfBase["dir"]:
    count=0
    for root,dirs,files in os.walk(ele):
        for file in files:
            count+=1
    dfBase.loc[seek,"count"]=count
    seek+=1
#将初步统计结果保存
dfBase.to_csv("./遮挡文件路径.csv",index=False) 

In [ ]:
# """ 这部分已经完成 """
# dfAllFilePath = pd.read_csv("./all_image_path.csv")
# imageFilesList = dfAllFilePath["breed"].tolist()
# classToIndex = {
#     x: i for i, x in enumerate(dfAllFilePath["breed"].unique())
# }
# IndexToClass = {
#     i: x for i, x in enumerate(dfAllFilePath["breed"].unique())
# }
# normalize = transforms.Normalize(
#     mean=[0.485, 0.456, 0.406],
#     std=[0.229, 0.224, 0.225]
# )

# dsTransforms = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     normalize
# ])
# # 由于数据集不同种类图片数量相差太大，所以训练、验证、测试集的划分要按照不同品种来划分
# """
# 首先把40个不同品种的所有路径、品种分别保存到40个DataFrame中。然后40个DataFrame放到1个List中。
# 由于之前统计过不同种类的图片数量，所以直接用数量来对大DataFrame切片
# """

# breedsList = list(IndexToClass.values())
# dfBreedsNumber = pd.read_csv("./breeds_number.csv")
# dfBreedsDict = {}
# begin = 0
# end = 0
# for i in range(len(dfBreedsNumber)):
#     breedName = dfBreedsNumber.iat[i, 0]
#     end = dfBreedsNumber.iat[i, 2]+end
#     tempDf = dfAllFilePath[begin:end]
#     dfBreedsDict[breedName] = tempDf
#     begin = end

# # 用于存储53个品种的文件地址链接的字典已经创建完成，接下来就分别划分训练、验证、测试集，然后添加到分别添加到大的训练、验证、测试集中
# dfTrain = pd.DataFrame(columns=["path", "breed"])
# dfValidate = dfTrain.copy()
# dfTest = dfTrain.copy()
# for ele in list(dfBreedsDict.keys()):
#     # ele是DataFrame
#     tempDf = dfBreedsDict[ele]
#     tempTrain, tempValidate, tempTest = np.split(
#         tempDf.sample(frac=1, random_state=42),
#         [
#             int(0.6*len(tempDf)),
#             int(0.8*len(tempDf)),
#         ]
#     )
#     dfTrain = dfTrain.append(tempTrain, ignore_index=True)
#     dfValidate = dfValidate.append(tempValidate, ignore_index=True)
#     dfTest = dfTest.append(tempTest, ignore_index=True)


In [11]:
# """ 测试集以及划分出来了，接下来更改路径为本地文件路径（原本是服务器路径），然后复制到品种对应的文件夹  """
# # dfTest["moveTo"]=None
# testDatasetPath=r"./images/test_dateset/"
# for seek in range(len(dfTest)):
#     temp=dfTest.iat[seek,0]
#     # break
#     temp=temp.replace(r"/raid/hjy/",r"./images/")
#     # dfTest.iat[seek,1]=temp
#     # prefix,suffix=temp.split(".")
#     copyPath=testDatasetPath+dfTest.iat[seek,1]+"/"
#     shutil.copy(temp,copyPath)